In [1]:
# Import library-library
import os
import glob
import string

# Data Preparation and Preprocessing
import pandas as pd
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

# Word Embedding
import joblib
from sklearn.feature_extraction.text import TfidfVectorizer
import scipy.sparse

from sklearn.metrics.pairwise import cosine_similarity

# Input and Expansion Query
from textblob import TextBlob
from nltk.tokenize import wordpunct_tokenize
#from googletrans import Translator

#Split to train and test
from sklearn.model_selection import train_test_split

import mysql.connector
from mysql.connector import errorcode

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\RayRoyy\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
excluded_words = []

NLTK_StopWords = stopwords.words('indonesian')
NLTK_StopWords.extend(["detik", "detikjatim", "detikjateng", "detikjabar", "detiksulsel", "detiksumbar", "detikbali", "detikpapua", "detiksulteng", "detikmaluku", "detjatim", "detikcom", "allahumma", "aamiin", "aamiin", "allah", "bismillah"])
NLTK_StopWords.extend(["yg", "dg", "rt", "dgn", "ny", "d", 'klo', 
                       'kalo', 'amp', 'biar', 'bikin', 'bilang', 
                       'gak', 'ga', 'krn', 'nya', 'nih', 'sih', 
                       'si', 'tau', 'tdk', 'tuh', 'utk', 'ya', 
                       'jd', 'jgn', 'sdh', 'aja', 'n', 't', 
                       'nyg', 'hehe', 'pen', 'u', 'nan', 'loh', 'rt',
                       '&amp', 'yah'])
txt_stopword = pd.read_csv("stopwords.txt", names= ["stopwords"], header = None)

# convert stopword string to list & append additional stopword
NLTK_StopWords.extend(txt_stopword["stopwords"][0].split(' '))
NLTK_StopWords = set(NLTK_StopWords)

In [3]:
def preprocessing(berita):
    # Preprocessing
    s = str(berita)
    s = s.lower()
    s = s.replace('\n', ' ')
    s = s.replace('\r', ' ')
    s = re.sub(r'[^a-zA-Z0-9\s]', ' ', s)
    tokens = [token for token in s.split(" ") if token != ""]

    # Stopwords checking
    T = [t for t in tokens if ((t in excluded_words) or (t not in NLTK_StopWords))]
    return T

In [4]:
# df =pd.read_csv("news.csv")
# df.info()
# df.head()
dataframe = []

try:
        cnx = mysql.connector.connect(user = 'root', password='Password', database = 'Petakabar')
        cursor = cnx.cursor()

        ambildata = ("SELECT berita_title, berita_date, berita_desc, berita_source FROM berita where berita_topik_id = 3 and class_classification is not null")
        cursor.execute(ambildata)
        dataa = cursor.fetchall()
        for row in dataa:
            dataframe.append(row)
        
except mysql.connector.Error as err:
        if err.errno == errorcode.ER_ACCESS_DENIED_ERROR:
            print("Something is wrong with your user name or password")
        elif err.errno == errorcode.ER_BAD_DB_ERROR:
            print("Database does not exist")
        else:
            print(err)
else:
    cursor.close()
    cnx.close()

In [5]:
import pandas as pd
writer = pd.DataFrame(dataframe, columns=[
                    'Title','Date','Description', 'Source'], index=None)
writer.to_csv('news.csv', index=False, sep=',')

In [6]:
df =pd.read_csv("news.csv")
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 922 entries, 0 to 921
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Title        922 non-null    object
 1   Date         922 non-null    object
 2   Description  922 non-null    object
 3   Source       922 non-null    object
dtypes: object(4)
memory usage: 28.9+ KB


,Title,Date,Description,Source
0,Bus Wisata Terjun ke Jurang Sedalam 30 Meter d...,"Senin, 05 Des 2022 10:44 WIB",Bus wisata yang berangkat dari Semarang kecela...,www.tribunnews.com
1,Kronologi Minibus Rombongan Pelayat Masuk Jura...,"Selasa, 29 Nov 2022 10:05 WIB",\- Kecelakaan maut minibus rombongan pelayat m...,www.tribunnews.com
2,Kendaraan Pembawa Bantuan Gempa Cianjur Kecela...,"Kamis, 08 Des 2022 10:30 WIB",Kecelakaan terjadi di Tol Cipali atau Cikopo-P...,www.tribunnews.com
3,Truk Boks Masuk Jurang Sedalam 100 Meter di Ga...,"Selasa, 06 Des 2022 11:08 WIB",Satu unit truk boks terjun ke jurang sedalam 1...,www.tribunnews.com
4,Dugaan Penyebab Tabrakan Kapal Roro MV Serasi ...,"Minggu, 04 Des 2022 09:32 WIB",Sebanyak 17 awak kapal roro MV Serasi I yang m...,www.kompas.com


In [7]:
column = ['Title', 'Date', 'Description', 'Source']
df_total = pd.DataFrame()

df = pd.read_csv("news.csv")
df = df.dropna(subset=['Description'], axis=0)
df.sort_values("Description", inplace = True)
df.drop_duplicates(subset ="Description", keep = False, inplace = True)

df.head()
df_train_unprocessed, df_test_unprocessed = train_test_split(df,test_size=0.1)

In [8]:
# df_first_1000 = df.head(1000)
# len(df_first_1000)
df_train, df_test = train_test_split(df,test_size=0.1)
len(df_train)


829

In [9]:
df_train.to_csv("corpus/df_train_unprerocessed.csv" , index = None, header=True)
df_test.to_csv("corpus/df_test_unprerocessed.csv" , index = None, header=True)

In [10]:
desc_text_test_unprocessed = []

for x in range(0, df_test_unprocessed.shape[0]):
  desc_text_test_unprocessed.append(df_test_unprocessed.iloc[x, -2])

joblib.dump(desc_text_test_unprocessed, "corpus/model/desc_text_test_unprocessed.pkl")

['corpus/model/desc_text_test_unprocessed.pkl']

In [11]:
# for x in range(0, df.shape[0]):
#     text = preprocessing(df.iloc[x, -2])
#     df.iloc[x, -2] = (' '.join(text)) 

desc_text_train_unpreprocessed = []
desc_text_test_unpreprocessed = []

for x in range(0, df_train.shape[0]):
  desc_text_train_unpreprocessed.append(df_train.iloc[x, -2])

for x in range(0, df_test.shape[0]):
  desc_text_test_unpreprocessed.append(df_test.iloc[x, -2])

In [12]:
# Simpan teks asli dokumen
joblib.dump(desc_text_test_unpreprocessed, "corpus/model/desc_text_test_unpreprocessed.pkl")
joblib.dump(desc_text_train_unpreprocessed, "corpus/model/desc_text_train_unpreprocessed.pkl")

# Simpan dataframe hasil text processing sebelum TF-IDF Vectorizer
df.to_csv('corpus/dataset/df_total_unpreprocessed.csv', index=False)
df_test.to_csv('corpus/dataset/df_test_unpreprocessed.csv', index=False)
df_train.to_csv('corpus/dataset/df_train_unpreprocessed.csv', index=False)


In [13]:
for x in range(0, df.shape[0]):
    text = preprocessing(df.iloc[x, -2])
    df.iloc[x, -2] = (' '.join(text)) 

In [14]:
desc_text_train = []
desc_text_test = []

for x in range(0, df_train.shape[0]):
  desc_text_train.append(df_train.iloc[x, -2])

for x in range(0, df_test.shape[0]):
  desc_text_test.append(df_test.iloc[x, -2])

In [15]:
# print(df_test.info())
print("Panjang Dokumen Test : ", len(desc_text_test))

print ('-'*90)

# print(df_train.info())
print("Panjang Dokumen Train : ", len(desc_text_train))

Panjang Dokumen Test :  93
------------------------------------------------------------------------------------------
Panjang Dokumen Train :  829


In [16]:
# Simpan teks asli dokumen
joblib.dump(desc_text_test, "corpus/model/desc_text_test.pkl")
joblib.dump(desc_text_train, "corpus/model/desc_text_train_new.pkl") #INI

# Simpan dataframe hasil text processing sebelum TF-IDF Vectorizer
df.to_csv('corpus/dataset/df_total_new.csv', index=False) #INI
df_test.to_csv('corpus/dataset/df_test.csv', index=False)
df_train.to_csv('corpus/dataset/df_train.csv', index=False)


In [17]:
# Fit transform menggunakan TF-IDF Vectorizer
tfidf_vectorizer = TfidfVectorizer()
tfidf_test_matrix = tfidf_vectorizer.fit_transform(desc_text_test)
tfidf_train_matrix = tfidf_vectorizer.fit_transform(desc_text_train)

print('tfidf_test_matrix')
print(tfidf_test_matrix.shape)
print('tfidf_train_matrix')
print(tfidf_train_matrix.shape)

tfidf_test_matrix
(93, 4103)
tfidf_train_matrix
(829, 16472)


In [18]:
joblib.dump(tfidf_vectorizer, "corpus/vectorizer/vectorizer_new.pkl") #INI
joblib.dump(tfidf_test_matrix, "corpus/matrix/tfidf_test.pkl")
joblib.dump(tfidf_train_matrix, "corpus/matrix/tfidf_train_new.pkl") #INI

['corpus/matrix/tfidf_train_new.pkl']

### Pencarian 

In [19]:
df_total = pd.read_csv('corpus/dataset/df_total_new.csv')
df_total = df_total[pd.notnull(df_total['Description'])]
print(df_total.info())
print ('-'*90)

document_text_train= joblib.load('corpus/model/desc_text_train_new.pkl')
document_text_test= joblib.load('corpus/model/desc_text_test.pkl')
print(len(document_text_test))
print(len(document_text_train))

<class 'pandas.core.frame.DataFrame'>
Int64Index: 922 entries, 0 to 921
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Title        922 non-null    object
 1   Date         922 non-null    object
 2   Description  922 non-null    object
 3   Source       922 non-null    object
dtypes: object(4)
memory usage: 36.0+ KB
None
------------------------------------------------------------------------------------------
93
829


In [23]:
kueri='meninggal'
kueri=preprocessing(kueri)
kueri= [" ".join (kueri)]
print (kueri)

tfidf_matrix =joblib.load( "corpus/matrix/tfidf_train_new.pkl" )
tfidf_vectorizer = joblib.load( "corpus/vectorizer/vectorizer_new.pkl" ) 
query_vec= tfidf_vectorizer.transform(kueri)
results=cosine_similarity(tfidf_matrix, query_vec).reshape((-1))
#print (results)
print("\n======================\n")
print("Top 10 most similar documents in corpus:")

j = 1

for i in results.argsort()[-10:][::-1]:
    print("No ID Dokumen  : ", i)
    print("Tanggal        : ", df_total.iloc[i,1])
    print("Isi berita     : ", df_total.iloc[i,2])
    print("(Score: %.4f) " % results[i])

['meninggal']


Top 10 most similar documents in corpus:
No ID Dokumen  :  595
Tanggal        :  Kamis, 22 Des 2022 03:00 WIB
Isi berita     :  ruas jalan nasional pantura karawang rusak dikeluhkan warga salah parah berlokasi jalan raya kosambi klari desa duren kecamatan klari kabupaten karawang ano 50 warga pengguna rutin jalan mengeluhkan kerusakan jalan pasalnya menyaksikan kecelakaan lintas akibat jalan rusak membahayakan kang pagi sepeda motor terguling terpepet lubang ruas jalan sebelah kanan ano diwawancara lokasi rabu 21 12 2022 kondisi jalan berlubang parah jalan nasional mayoritas berlokasi persimpangan jalan putar berlubang putar arah kaya deket putaran persimpangan rawan mobil ngrem kecelakaan menyaksikan kecelakaan kendaraan seputar jalan rusak kecelakaan mobil kenceng jalan berlubang ngerem mendadak sepeda motor nggak ngrem nabrak mobil terjadj dikonfirmasi terpisah kepala bidang jalan jembatan dinas pekerjaan penataan ruang pupr kabupaten karawang chris priyanto jalan na